#  Segmenting and Clustering Neighborhoods in Toronto
In this Notebook I will gather information about Toronto neighborhoods from different sources (Wikipedia, Foursquares, etc.). I will use different Python libraries to gather and clean the data. From this new dataset I will |train a model (K nearsest neighbors) to cluster the neighborhoods based on the venues in that area. Then we will plot these finding on maps and figures to illustrate their meaning. 

## Part 1 - Scrapping Wikipedia and creating dataframe

In this part we will gather data from the wikipedia website on Torronto. The postal code, neighborhood and borough will be stored in a dataframe. 

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, 

In [97]:
# importing libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from bs4 import BeautifulSoup
import requests as req

import json # library to handle JSON files
from pandas import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# map rendering library folium
#!conda install -c conda-forge Folium=0.5.0 --yes 
import folium 

print('libraries imported succesfully')

libraries imported succesfully


In [98]:
# making a get request from wikipedia and storing the HTML data in beautifullsoup object
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_doc = req.get(url).text
soup = BeautifulSoup(html_doc, 'html5lib')

In [99]:

#To turn al the cells of the table into a list to iterate through
table_cells = soup.table.find_all('td')

#create a empty pandas dataframe to store the information
df_postal = pd.DataFrame()

# iterate through the cells to extract the data
for cells in table_cells:
    PostalCode = cells.b.text
    Borough_Neighborhood = cells.span.text
    
    try: #not all PostalCodes have a borough or neighborhoods associated with them 
        Borough = Borough_Neighborhood.split("(")[0]
        try:
            Neighborhood = Borough_Neighborhood.split("(")[1].replace(")", " ").replace('/', ',') + ", " + Borough_Neighborhood.split("(")[2].replace(")", " ").replace('/', ',')
        except:
            Neighborhood = Borough_Neighborhood.split("(")[1].replace(")", " ").replace('/', ',')
    except:
        Borough = 'NA'
        Neighborhood = 'NA'
    
    df_temp = pd.DataFrame(data=[[PostalCode, Borough, Neighborhood]])
    df_postal = pd.concat([df_postal, df_temp])
    
# cleaning dataframe
df_postal.rename(columns = {0:'Postal Code', 1:'Borough', 2:'Neighborhood'}, inplace = True) #name columns
df_postal = df_postal[df_postal.Borough != 'NA'] #drop all rows without bourough information
df_postal.reset_index(drop = True, inplace=True)

### review of dataframe
The data frame has 103 PostalCodes which is the same number Wikipedia suggest there are in Torronto. We can distinguish 15 different Boruoughs in the dataframe, some of which seem to be Mail processing centres we might nog use later on the analyses.

In [100]:
# check dataframe
print('there are ' + str(len(df_postal['Postal Code'])) + ' distinct Postal Codes in our dataframe')

print('From ' + str(len(pd.unique(df_postal['Borough']))) + ' different Boroughs')
print("")
print(pd.unique(df_postal['Borough']))


there are 103 distinct Postal Codes in our dataframe
From 15 different Boroughs

['North York' 'Downtown Toronto' "Queen's Park" 'Etobicoke' 'Scarborough'
 'East York' 'York' 'East Toronto' 'West Toronto' 'East YorkEast Toronto'
 'Central Toronto' 'MississaugaCanada Post Gateway Processing Centre'
 'Downtown TorontoStn A PO Boxes25 The Esplanade' 'EtobicokeNorthwest'
 'East TorontoBusiness reply mail Processing Centre969 Eastern']


In [101]:
df_postal.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Part 2 -  Importing coordinates for each postal code

### Plan A - geocoder
First I will try to get the coordinates using the geocoder. Unfortunately I was unable to get the coordinates using the geocoder. I only received a feedback of None every single time. 

%pip install geocoder
import geocoder# import geocoder
print('succesfully imported geocoder')

lat_lng_coords = None

while(lat_lng_coords is None):
    g = geocoder.google('M3A, Toronto, Ontario') ## Test for one specific Postal code 
    lat_lng_coords = g.latlng
    print(lat_lng_coords) ## review the feedback from the geocoder (which gives a None value consistently)

df_coordinates = pd.DataFrame() #create empty df

for PC in df_postal['PostalCode']:   ## Trying to loop through all the postal codes in the dataframe
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):


        g = geocoder.google('{},Toronto, Ontario'.format(PC))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)## review the feedback from the geocoder (which gives a None value consistently)

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]

    #Store data in dataframe
    df_temp_coordinates = pd.DataFrame(data=[[PC, latitude, longitude]])
    df_coordinates = pd.concat([df_coordinates, df_temp_coordinates])
    print(Pc, latitude, longitude)
    
df_coordinates.head()
    

### Plan B - using CSV
Using the csv file with coordinates to build dataframe


In [6]:
path = "https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv"
df_coor = pd.read_csv(path)
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [18]:
df_toronto = pd.merge(df_postal, df_coor, on='Postal Code')
df_toronto.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


# Part 3 - Maping and clustering our Neighborhoods
To get a feel for the different neighborhoods I will start with plotting them on a map using Folium.


In [91]:
# finding the centre of our data points to zoom into on the map
latitude_toronto = df_toronto['Latitude'].mean()
longitude_toronto = df_toronto['Longitude'].mean()

print('coordinates of toronto are: ', latitude_toronto, longitude_toronto)


coordinates of toronto are:  43.704607733980595 -79.39715291165048


In [14]:
# create map of Toronto using latitude and longitude values

map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Using foursquare to get data for clustering
Below I will make some call to the foursquare API to retrieve venue data for all of our neighborhoods. This will be stored in a data frame that can function as an input for KNN clustering algorithm. 

### Defining credentials Foursquare

In [212]:
# The code was removed by Watson Studio for sharing.

### Defining function to get the data from foursquare

In [213]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = req.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Making API call to Foursquare and building the dataframe with the defined function above

In [214]:
toronto_venues = getNearbyVenues(df_toronto['Neighborhood'], df_toronto['Latitude'], df_toronto['Longitude'], radius)


Parkwoods 
Victoria Village 
Regent Park , Harbourfront 
Lawrence Manor , Lawrence Heights 
Ontario Provincial Government 
Islington Avenue 
Malvern , Rouge 
Don Mills North
Parkview Hill , Woodbine Gardens 
Garden District, Ryerson 
Glencairn 
West Deane Park , Princess Gardens , Martin Grove , Islington , Cloverdale 
Rouge Hill , Port Union , Highland Creek 
Don Mills South, Flemingdon Park 
Woodbine Heights 
St. James Town 
Humewood-Cedarvale 
Eringate , Bloordale Gardens , Old Burnhamthorpe , Markland Wood 
Guildwood , Morningside , West Hill 
The Beaches 
Berczy Park 
Caledonia-Fairbanks 
Woburn 
Leaside 
Central Bay Street 
Christie 
Cedarbrae 
Hillcrest Village 
Bathurst Manor , Wilson Heights , Downsview North 
Thorncliffe Park 
Richmond , Adelaide , King 
Dufferin , Dovercourt Village 
Scarborough Village 
Fairview , Henry Farm , Oriole 
Northwood Park , York University 
The Danforth  East 
Harbourfront East , Union Station , Toronto Islands 
Little Portugal , Trinity 
Kennedy

# Exploring the dataframe containing all the venues


In [215]:
print(toronto_venues.shape)
toronto_venues.head(10)

(2123, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
5,Victoria Village,43.725882,-79.315572,Pizza Nova,43.725824,-79.312860,Pizza Place
6,"Regent Park , Harbourfront",43.654260,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
7,"Regent Park , Harbourfront",43.654260,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
8,"Regent Park , Harbourfront",43.654260,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
9,"Regent Park , Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [216]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood , Long Branch",9,9,9,9,9,9
"Bathurst Manor , Wilson Heights , Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
Berczy Park,59,59,59,59,59,59
"Birch Cliff , Cliffside West",4,4,4,4,4,4
"Brockton , Parkdale Village , Exhibition Place",24,24,24,24,24,24
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",16,16,16,16,16,16


In [217]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 275 uniques categories.


# Preparing Dataframe for KNN
using one hot encoding

In [218]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot ['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
toronto_onehot.set_index('Neighborhood', inplace=True)
toronto_onehot.reset_index(inplace=True)

#display head
toronto_onehot.head(12)

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [219]:

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean()
toronto_grouped.reset_index(inplace = True)

print(toronto_grouped.shape)
toronto_grouped.head()


(100, 275)


,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.

### Display most common venues


In [220]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [277]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Lounge,Clothing Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop
1,"Alderwood , Long Branch",Pizza Place,Pub,Sandwich Place,Gym,Skating Rink,Dance Studio,Coffee Shop,Athletics & Sports,Moroccan Restaurant,Monument / Landmark
2,"Bathurst Manor , Wilson Heights , Downsview No...",Coffee Shop,Bank,Bridal Shop,Frozen Yogurt Shop,Fried Chicken Joint,Restaurant,Gas Station,Diner,Sandwich Place,Intersection
3,Bayview Village,Café,Japanese Restaurant,Chinese Restaurant,Bank,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Men's Store
4,"Bedford Park , Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Butcher,Toy / Game Store,Juice Bar,Sushi Restaurant,Liquor Store,Pub,Café


# KNN clustering
## 

### Running the KNN algorithm

In [297]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 4, 4, 4, 4, 4, 0], dtype=int32)

### Constructing the new combined dataframe with all inputs for plotting map

In [298]:
#Starting out with the postal code and neighborhood data from wikipedia
toronto_merged = df_toronto
print('shape before', toronto_merged.shape)

# add clustering labels from KNN 
try: 
    neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)
except:
    neighborhoods_venues_sorted['Cluster_Labels'] = kmeans.labels_

# merging neighborhood data with venue data from foursquare 
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').dropna()  # drops neigborhoods without any available venue information and thus no cluster data

# change data type of clusters to int to avoid error plotting map
toronto_merged ['Cluster_Labels'] = toronto_merged.Cluster_Labels.astype(int) 


print('shape after', toronto_merged.shape)
print('There were 3 neighborhoods without any venue information')
toronto_merged.head(20)

shape before (103, 5)
shape after (100, 16)
There were 3 neighborhoods without any venue information


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Food & Drink Shop,Park,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Accessories Store
1,M4A,North York,Victoria Village,43.725882,-79.315572,4,Pizza Place,Coffee Shop,Hockey Arena,Portuguese Restaurant,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,4,Coffee Shop,Park,Bakery,Pub,Café,Breakfast Spot,Theater,Restaurant,Yoga Studio,Spa
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Accessories Store,Arts & Crafts Store,Coffee Shop,Carpet Store,Sporting Goods Shop,Boutique,Vietnamese Restaurant,Music Venue
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494,4,Coffee Shop,Sushi Restaurant,College Cafeteria,Beer Bar,Smoothie Shop,Burrito Place,Café,Sandwich Place,Restaurant,College Auditorium
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,4,Fast Food Restaurant,Middle Eastern Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Pizza Place
7,M3B,North York,Don Mills North,43.745906,-79.352188,4,Japanese Restaurant,Caribbean Restaurant,Gym,Café,Molecular Gastronomy Restaurant,Moving Target,Movie Theater,Motel,Moroccan Restaurant,Monument / Landmark
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937,4,Pizza Place,Athletics & Sports,Pharmacy,Café,Bank,Gym / Fitness Center,Breakfast Spot,Flea Market,Gastropub,Intersection
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,4,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Cosmetics Shop,Middle Eastern Restaurant,Electronics Store,Ramen Restaurant,Burger Joint,Lingerie Store
10,M6B,North York,Glencairn,43.709577,-79.445073,4,Japanese Restaurant,Asian Restaurant,Sushi Restaurant,Bakery,Accessories Store,Mobile Phone Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant


# Visualising the clusters on a map

In [296]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.8).add_to(map_clusters)
       
map_clusters